# Spoiler generation

In [9]:
import pandas as pd
from datasets import load_dataset
from copy import deepcopy

from spoiler_generation.utils.stats import prepare_stats, calculate_bleu
from spoiler_generation.utils.dataset_class import Dataset

In [ ]:
# The dataset is private
dataset = load_dataset("MateuszW/clickbait_spoiling_test")
test_df = pd.DataFrame(dataset["test"])
test_df["spoiler"] = test_df["spoiler"].apply(Dataset.preprocess_func)

In [31]:
test_df.shape

(1000, 6)

In [ ]:
models_outputs = load_dataset(
    "MateuszW/spoiler_generation",
    data_files={
        "baseline": "models_output/deberta-baseline_output.csv",
        "deberta_paqott": "models_output/deberta-paqott_output.csv",
        "llama_pwot": "models_output/llama-pwot_output.csv",
        "vicuna_pwot": "models_output/vicuna-pwot_output.csv",
        "opt_pwot": "models_output/opt-pwot_output.csv",
        "llama_pwt": "models_output/llama-pwt_output.csv",
        "vicuna_pwt": "models_output/vicuna-pwt_output.csv",
        "opt_pwt": "models_output/opt-pwt_output.csv",
        "vicuna_ppt": "models_output/vicuna-ppt_output.csv",
        "best": "models_output/best_output.csv",
    },
)

In [11]:
for output_name in models_outputs.keys():
    models_outputs[output_name] = pd.DataFrame(models_outputs[output_name])
    models_outputs[output_name]["spoiler"] = models_outputs[output_name]["spoiler"].apply(Dataset.preprocess_func)

In [13]:
for output_name in models_outputs.keys():
    print(output_name, round(calculate_bleu(test_df, models_outputs[output_name]), 4))

baseline 0.3789
deberta_paqott 0.3821
llama_pwot 0.3221
vicuna_pwot 0.3316
opt_pwot 0.3421
llama_pwt 0.377
vicuna_pwt 0.388
opt_pwt 0.3386
vicuna_ppt 0.4001
best 0.4437


### Per type

In [21]:
for spoiler_type in ["phrase", "passage", "multi"]:
    spoiler_type_df = deepcopy(test_df.loc[test_df["type"] == spoiler_type])
    print(spoiler_type, round(calculate_bleu(spoiler_type_df, pd.DataFrame(models_outputs["best"])), 4))

phrase 0.6457
passage 0.2996
multi 0.2866
